In [45]:
# HYPOTHESIS AND INFERENCE
# the goal is to test whether a certain hypothesis is likely to be true.
# Example: is a coin fair? you compare the null hypothesis H0 to an alternative hypothesis. Fair coin p=0.5, hypothesis p!=0.5
from IPython.display import clear_output
from importnb import Notebook
prob = Notebook.load("6 probability.ipynb")
clear_output() # work around to load ipynb file

In [46]:
from typing import Tuple
import math

def normal_approximation_to_binomial(n: int, p: float) -> Tuple[float, float]:
    """Returns mu and sigma corresponding to a Binomial(n, p)"""
    mu = p * n
    sigma = math.sqrt(p * (1 - p) * n)
    return mu, sigma

# The normal cdf _is_ the probability the variable is below a threshold
normal_probability_below = prob.normal_cdf

# It's above the threshold if it's not below the threshold
def normal_probability_above(lo: float,
                             mu: float = 0,
                             sigma: float = 1) -> float:
    """The probability that an N(mu, sigma) is greater than lo."""
    return 1 - prob.normal_cdf(lo, mu, sigma)

# It's between if it's less than hi, but not less than lo
def normal_probability_between(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """The probability that an N(mu, sigma) is between lo and hi."""
    return prob.normal_cdf(hi, mu, sigma) - prob.normal_cdf(lo, mu, sigma)

# It's outside if it's not between
def normal_probability_outside(lo: float,
                               hi: float,
                               mu: float = 0,
                               sigma: float = 1) -> float:
    """The probability that an N(mu, sigma) is not between lo and hi."""
    return 1 - normal_probability_between(lo, hi, mu, sigma)

In [47]:
def normal_upper_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Returns the z for which P(Z <= z) = probability"""
    return prob.inverse_normal_cdf(probability, mu, sigma)

def normal_lower_bound(probability: float,
                       mu: float = 0,
                       sigma: float = 1) -> float:
    """Returns the z for which P(Z >= z) = probability"""
    return prob.inverse_normal_cdf(1 - probability, mu, sigma)

def normal_two_sided_bounds(probability: float,
                            mu: float = 0,
                            sigma: float = 1) -> Tuple[float, float]:
    """
    Returns the symmetric (about the mean) bounds
    that contain the specified probability
    """
    tail_probability = (1 - probability) / 2

    # upper bound should have tail_probability above it
    upper_bound = normal_lower_bound(tail_probability, mu, sigma)

    # lower bound should have tail_probability below it
    lower_bound = normal_upper_bound(tail_probability, mu, sigma)

    return lower_bound, upper_bound

In [48]:
mu_0, sigma_0 = normal_approximation_to_binomial(1000, 0.5)
print(mu_0, sigma_0)
# 95% bounds based on assumption p is 0.5
lo, hi = normal_two_sided_bounds(0.95, mu_0, sigma_0)
print(lo, hi)

# let's check what happens if p is 0.55
# actual mu and sigma based on p = 0.55
mu_1, sigma_1 = normal_approximation_to_binomial(1000, 0.55)
print(mu_1, sigma_1)

500.0 15.811388300841896
469.01026640487555 530.9897335951244
550.0 15.732132722552274


In [49]:
# a type 2 error means we fail to reject the null hypothesis,
# which will happen when X is still in our original interval
type_2_probability = normal_probability_between(lo, hi, mu_1, sigma_1)
power = 1 - type_2_probability
print(power)

hi = normal_upper_bound(0.95, mu_0, sigma_0)
# is 526 (< 531, since we need more probability in the upper tail)

type_2_probability = normal_probability_below(hi, mu_1, sigma_1)
power = 1 - type_2_probability   
print(power)    # 0.936

0.8865480012953671
0.9363794803307173


In [50]:
def two_sided_p_value(x: float, mu: float = 0, sigma: float = 1) -> float:
    """
    How likely are we to see a value at least as extreme as x (in either
    direction) if our values are from an N(mu, sigma)?
    """
    if x >= mu:
        # x is greater than the mean, so the tail is everything greater than x
        return 2 * normal_probability_above(x, mu, sigma)
    else:
        # x is less than the mean, so the tail is everything less than x
        return 2 * normal_probability_below(x, mu, sigma)

# we want to see 530 heads(2 sides). We put 529.5 for continuity correction
two_sided_p_value(529.5, mu_0, sigma_0)   # 0.062

0.06207721579598835

In [52]:
import random

# to prove that our logic is working we can do a simulation
extreme_value_count = 0
for _ in range(1000):
    num_heads = sum(1 if random.random() < 0.5 else 0    # Count # of heads
                    for _ in range(1000))                # in 1000 flips,
    if num_heads >= 530 or num_heads <= 470:             # and count how often
        extreme_value_count += 1                         # the # is 'extreme'

# p-value was 0.062 => ~62 extreme values out of 1000
assert 59 <= extreme_value_count <= 65, f"{extreme_value_count}"

In [ ]:
# if we have a result < than the significance (5%) we should discard the H0
print(two_sided_p_value(531.5, mu_0, sigma_0))

# same for upper and lower p
upper_p_value = normal_probability_above
lower_p_value = normal_probability_below
print(upper_p_value(524.5, mu_0, sigma_0)) # keep
print(upper_p_value(526.5, mu_0, sigma_0)) # discard

0.046345287837786575
0.06062885772582072
0.04686839508859242


In [ ]:
p_hat = 525 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)   # 0.0158
print(normal_two_sided_bounds(0.95, mu, sigma)) # [0.4940, 0.5560]

# it's still fair because it stays in our confidence interval.
# if we increase it to 540 it gets too influenced to be fair
p_hat = 540 / 1000
mu = p_hat
sigma = math.sqrt(p_hat * (1 - p_hat) / 1000)   # 0.0158
print(normal_two_sided_bounds(0.95, mu, sigma)) # [0.5091, 0.5709]

(0.4940490278129096, 0.5559509721870904)
(0.5091095927295919, 0.5708904072704082)


In [ ]:
# p-hacking -> you test enough hypotheses against your dataset. you remove the outliers and you can get your p-value below 0.05
from typing import List

def run_experiment() -> List[bool]:
    """Flips a fair coin 1000 times, True = heads, False = tails"""
    return [random.random() < 0.5 for _ in range(1000)]

def reject_fairness(experiment: List[bool]) -> bool:
    """Using the 5% significance levels"""
    num_heads = len([flip for flip in experiment if flip])
    return num_heads < 469 or num_heads > 531

random.seed(0)
experiments = [run_experiment() for _ in range(1000)]
num_rejections = len([experiment
                      for experiment in experiments
                      if reject_fairness(experiment)])

assert num_rejections == 46

In [ ]:
# A/B testing
def estimated_parameters(N: int, n: int) -> Tuple[float, float]:
    p = n / N
    sigma = math.sqrt(p * (1 - p) / N)
    return p, sigma
def a_b_test_statistic(N_A: int, n_A: int, N_B: int, n_B: int) -> float:
    p_A, sigma_A = estimated_parameters(N_A, n_A)
    p_B, sigma_B = estimated_parameters(N_B, n_B)
    return (p_B - p_A) / math.sqrt(sigma_A ** 2 + sigma_B ** 2)

z = a_b_test_statistic(1000, 200, 1000, 180)    # -1.14
#probability we would see these clicks numbers if the ads were equally effective
print(two_sided_p_value(z))                     # 0.254

z = a_b_test_statistic(1000, 200, 1000, 150)    # -2.94
# here the probability gets lower and lower
print(two_sided_p_value(z))                     # 0.003

In [ ]:
# Bayesian Inference
def B(alpha: float, beta: float) -> float:
    """A normalizing constant so that the total probability is 1"""
    return math.gamma(alpha) * math.gamma(beta) / math.gamma(alpha + beta)

def beta_pdf(x: float, alpha: float, beta: float) -> float:
    if x <= 0 or x >= 1:          # no weight outside of [0, 1]
        return 0
    return x ** (alpha - 1) * (1 - x) ** (beta - 1) / B(alpha, beta)

# you start with an hypothesis: a fair coin is 55-45. Then you test it and get data that adds up to it.
# you lunch a coin 10 times and you get 5-5 so you add them 60-55. You can repeat it, the more you repeat the less the first assumption will weight.
# the goal is to have a statement easy to understand but still statistically powerfull like: 
# “Based on the prior and the observed data, there is only a 5% likelihood the coin’s heads probability is between 49% and 51%.”